# ministry Ministry Cybersecurity Assessment

This notebook uses the `cyber_relevance_assessment.py` script to automatically assess whether ministries across different countries are direct stakeholders in their country's cybersecurity ecosystem.

## Dataset
- **Source**: `ministries_of_ministry_done_with_all_countries.xlsx`
- **Domain**: Ministries/departments
- **Countries**: 193 countries with ministry ministry data

## Assessment Goal
Determine if each country's ministry ministry is:
- **High relevance**: Directly responsible for cybersecurity in ministry sector
- **Medium relevance**: Some cybersecurity involvement/coordination role
- **Low relevance**: Minimal cybersecurity involvement
- **No relevance**: No cybersecurity responsibilities

In [1]:
# Import required libraries
import pandas as pd
import asyncio
from pathlib import Path
import sys
import os

# Import the cyber relevance assessment functions
from cyber_relevance_assessment import (
    CyberStakeholderAssessment,
    CyberRelevanceLevel,
    ConfidenceLevel,
    load_country_data,
    run_cyber_assessment,
    process_results,
    save_results,
    display_sample_results
)

# Configuration: Set to True for quick testing (5 countries), False for full assessment (all countries)
TEST_MODE = False

print("✅ Successfully imported cyber relevance assessment tools")
print(f"📋 Configuration: {'TEST MODE (5 countries)' if TEST_MODE else 'FULL ASSESSMENT MODE (all countries)'}")

✅ Successfully imported cyber relevance assessment tools
📋 Configuration: FULL ASSESSMENT MODE (all countries)


In [2]:
# Load and explore the  ministry dataset
data_file = '../data/ministries_of_transportation_done_with_all_countries.xlsx'
domain = 'transportation'
ministry_data = pd.read_excel(data_file)

print(f"Dataset shape: {ministry_data.shape}")
print(f"Columns: {list(ministry_data.columns)}")
print(f"\nUnique countries: {ministry_data['COUNTRY'].nunique()}")
print(f"Total records: {len(ministry_data)}")

# Show sample data
print("\nSample data:")
ministry_data[['COUNTRY']].head(10)

Dataset shape: (193, 32)
Columns: ['SCOID', 'ISO3', 'COUNTRY', 'ORGANIZATION', 'POSITION', 'MINISTRY_OF_TRANSPORTATION', 'EXISTING', 'INTEL_ORG', 'MIL_ORG', 'CIL_ORG', 'POLICY_ORG', 'TECHNICAL_ORG', 'INTERNAL/EXTERNAL', 'CYB_CRIME', 'CIP', 'TERRORISM', 'E-GOVERNANCE', 'ORGANIZATION_NAME', 'ORG_NAME_NATIVE', 'IN_PROCESS', 'MONTH_CR', 'YEAR_CR', 'MONTH_OPERATIONAL', 'YEAR_OPERATIONAL', 'YEAR_TERMINATED', 'REPLACED_BY (AS OF 2025)', 'PARENT_ORG', 'CODER 1', 'CODER 2', 'CODER 3', 'CODER 4', 'CODER 5']

Unique countries: 193
Total records: 193

Sample data:


,COUNTRY
0,Afghanistan
1,Albania
2,Algeria
3,Andorra
4,Angola
5,Antigua and Barbuda
6,Argentina
7,Armenia
8,Australia
9,Austria


In [3]:
# Extract unique countries for assessment
countries = ministry_data['COUNTRY'].unique().tolist()
print(f"Countries to assess: {len(countries)}")
print(f"First 10 countries: {countries[:10]}")
print(f"Last 10 countries: {countries[-10:]}")

Countries to assess: 193
First 10 countries: ['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria']
Last 10 countries: ['United Republic of Tanzania', 'United States of America', 'Uruguay', 'Uzbekistan', 'Vanuatu', 'Venezuela, Bolivarian Republic of', 'Vietnam', 'Yemen', 'Zambia', 'Zimbabwe']


In [4]:
# Run cybersecurity assessment based on TEST_MODE configuration
if TEST_MODE:
    # Test mode: run on first 2 countries only
    assessment_countries = countries[:5]
    max_queries = 3
    filename_suffix = "test"
    print(f"🧪 TEST MODE: Running assessment on {len(assessment_countries)} countries: {assessment_countries}")
else:
    # Full mode: run on all countries
    assessment_countries = countries
    max_queries = None
    filename_suffix = "full_assessment"
    print(f"🚀 FULL MODE: Running assessment on {len(assessment_countries)} countries...")
    print("This will take approximately 3-4 minutes...")

# Run the assessment
# Now with true concurrency - max_concurrent > 1 will process queries in parallel
results = await run_cyber_assessment(
    domain=domain,
    countries=assessment_countries,
    max_queries=max_queries,
    delay_seconds=0.5,  # Shorter delay for concurrent processing
    max_concurrent=1    # Process up to 3 queries simultaneously
)

print(f"\n✅ Assessment completed! Generated {len(results)} assessments")

🚀 FULL MODE: Running assessment on 193 countries...
This will take approximately 3-4 minutes...
Starting cybersecurity stakeholder assessment for transportation ministry...
Assessing 193 countries
Creating spray with 193 word combinations and 1 research questions...
Total API calls to make: 193
Execution mode: sequential (max_concurrent=1)

Processing query 1/193: {'domain': 'transportation', 'country': 'Afghanistan'}
  Query 1/193: Is the department/ministry of transportation in Afghanistan a direct stakeholder...
    ✓ Got structured response (1165 chars, 0 retries)

Processing query 2/193: {'domain': 'transportation', 'country': 'Albania'}
  Query 2/193: Is the department/ministry of transportation in Albania a direct stakeholder (i....
    ✓ Got structured response (1644 chars, 0 retries)

Processing query 3/193: {'domain': 'transportation', 'country': 'Algeria'}
  Query 3/193: Is the department/ministry of transportation in Algeria a direct stakeholder (i....
    ✓ Got structured 

In [5]:
# Display assessment results
num_samples = 5 if TEST_MODE else 10
display_sample_results(results, num_samples=num_samples)

# Show the structured data preview
print("\n" + "="*60)
print("STRUCTURED DATA PREVIEW")
print("="*60)
print(results[['country', 'structured_data']].head())


SAMPLE RESULTS

--- Afghanistan ---
Relevance Level: CyberRelevanceLevel.LOW
Confidence: ConfidenceLevel.MEDIUM
Explanation: The Ministry of Communications and Information Technology (MCIT) is identified as the primary government body responsible for Afghanistan's cybersecur...

--- Albania ---
Relevance Level: CyberRelevanceLevel.MEDIUM
Confidence: ConfidenceLevel.MEDIUM
Explanation: The Ministry of Infrastructure and Energy in Albania, which includes transportation responsibilities, is considered part of the critical information i...

--- Algeria ---
Relevance Level: CyberRelevanceLevel.LOW
Confidence: ConfidenceLevel.HIGH
Explanation: The primary responsibility for cybersecurity in Algeria lies with the Information Systems Security Agency (ASSI) under the Ministry of National Defenc...

--- Andorra ---
Relevance Level: CyberRelevanceLevel.LOW
Confidence: ConfidenceLevel.MEDIUM
Explanation: Available information on Andorra's government and ministries indicates that the Ministry or D

In [ ]:
# Process and examine the results structure
processed_results = process_results(results)

def process_results(spray_output: pd.DataFrame) -> pd.DataFrame:
    """
    Process spray output to create a clean DataFrame with expanded structured data and prettified citations.
    
    Args:
        spray_output: Raw output from the sprayer
        
    Returns:
        Processed DataFrame with expanded structured data columns and prettified citations
    """
    import ast
    
    def safe_parse_citations(citations_str: str) -> List[Dict[str, Any]]:
        """Safely parse citations string from Python dictionary format."""
        if pd.isna(citations_str) or not citations_str:
            return []
        
        # If it's already a list, return it
        if isinstance(citations_str, list):
            return citations_str
        
        citations_str = str(citations_str).strip()
        
        try:
            # Use ast.literal_eval for Python dictionary strings
            return ast.literal_eval(citations_str)
        except (ValueError, SyntaxError):
            print(f"Warning: Could not parse citations: {citations_str[:100]}...")
            return []
    
    def format_citation(citation: Dict[str, Any], index: int) -> str:
        """Format a single citation into a human-readable string."""
        if not isinstance(citation, dict):
            return f"[{index}] Invalid citation format"
        
        # Extract citation information (excluding 'matched')
        url = citation.get('url', 'No URL')
        title = citation.get('title', 'No Title')
        snippet = citation.get('snippet', '')
        date = citation.get('date', '')
        last_updated = citation.get('last_updated', '')
        
        # Build formatted citation
        citation_parts = []
        
        # Add title and URL
        if title and title != 'No Title':
            citation_parts.append(f"**{title}**")
        
        if url and url != 'No URL':
            citation_parts.append(f"URL: {url}")
        
        # Add dates if available
        date_parts = []
        if date:
            date_parts.append(f"Published: {date}")
        if last_updated:
            date_parts.append(f"Updated: {last_updated}")
        if date_parts:
            citation_parts.append(" | ".join(date_parts))
        
        # Add snippet if available
        if snippet:
            # Clean up snippet (remove extra quotes, limit length)
            clean_snippet = snippet.strip().strip('"\'')
            if len(clean_snippet) > 150:
                clean_snippet = clean_snippet[:147] + "..."
            citation_parts.append(f"Snippet: {clean_snippet}")
        
        # Combine all parts
        formatted = f"[{index}] " + "\n    ".join(citation_parts)
        return formatted
    
    def prettify_citations(citations_str: str) -> str:
        """Convert citations string to prettified format."""
        citations = safe_parse_citations(citations_str)
        
        if not citations:
            return "No citations available"
        
        prettified = []
        for i, citation in enumerate(citations, 1):
            formatted_citation = format_citation(citation, i)
            prettified.append(formatted_citation)
        
        return "\n\n" + "\n\n".join(prettified)
    
    # Create a DataFrame from the processed results
    df = spray_output.copy()
    
    # Prettify enriched_citations if it exists
    if 'enriched_citations' in df.columns:
        df['enriched_citations'] = df['enriched_citations'].apply(prettify_citations)
    
    first_columns = df[['domain','country']]
    last_columns = df[['research_question', 'enriched_citations']]
    
    # Expand structured_data into separate columns if it exists
    if 'structured_data' in spray_output.columns:
        # Handle both Pydantic objects and dictionaries
        structured_series = spray_output['structured_data'].apply(
            lambda x: x.model_dump() if hasattr(x, 'model_dump') else x if isinstance(x, dict) else {}
        )
        structured_df = structured_series.apply(pd.Series)
        df = pd.concat([first_columns, structured_df, last_columns], axis=1)
    
    return df


print("Processed columns:", list(processed_results.columns))
print(f"\nProcessed data sample ({len(processed_results)} total records):")
processed_results.head()

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
# Save assessment results
filename_base = f"{domain}_cyber_relevance_{filename_suffix}"
save_results(results, filename_base, "output")
print(f"\n✅ Results saved as '{filename_base}'!")

Raw results saved to output/transportation_cyber_relevance_test.jsonl
Processed results saved to output/transportation_cyber_relevance_test.csv

✅ Results saved as 'transportation_cyber_relevance_test'!


## Results Analysis

Analysis of the assessment results based on the current mode.

In [ ]:
# Analyze results distribution
if 'relevance_level' in processed_results.columns:
    print("📊 CYBERSECURITY RELEVANCE DISTRIBUTION")
    print("="*50)
    relevance_counts = processed_results['relevance_level'].value_counts()
    print(relevance_counts)
    
    # Show percentage distribution
    relevance_pct = processed_results['relevance_level'].value_counts(normalize=True) * 100
    print("\nPercentage distribution:")
    for level, pct in relevance_pct.items():
        print(f"{level.upper()}: {pct:.1f}%")
        
# Show confidence distribution
if 'confidence' in processed_results.columns:
    print("\n📊 CONFIDENCE DISTRIBUTION")
    print("="*50)
    confidence_counts = processed_results['confidence'].value_counts()
    print(confidence_counts)

📊 CYBERSECURITY RELEVANCE DISTRIBUTION
relevance_level
CyberRelevanceLevel.LOW       2
CyberRelevanceLevel.MEDIUM    1
Name: count, dtype: int64

Percentage distribution:
LOW: 66.7%
MEDIUM: 33.3%

📊 CONFIDENCE DISTRIBUTION
confidence
ConfidenceLevel.HIGH      2
ConfidenceLevel.MEDIUM    1
Name: count, dtype: int64


In [ ]:
# Show countries with HIGH cybersecurity relevance
if 'relevance_level' in processed_results.columns:
    high_relevance = processed_results[processed_results['relevance_level'] == 'high']
    print(f"\n🔥 COUNTRIES WITH HIGH CYBERSECURITY RELEVANCE ({len(high_relevance)})")
    print("="*60)
    
    if len(high_relevance) > 0:
        for _, row in high_relevance.iterrows():
            print(f"• {row['country']} (Confidence: {row.get('confidence', 'N/A')})")
            if 'explanation' in row and row['explanation']:
                explanation = row['explanation'][:100] + "..." if len(row['explanation']) > 100 else row['explanation']
                print(f"  └─ {explanation}")
            print()
    else:
        print("No countries with HIGH relevance found in this assessment.")
        
    # Show medium relevance too if in test mode
    if TEST_MODE:
        medium_relevance = processed_results[processed_results['relevance_level'] == 'medium']
        if len(medium_relevance) > 0:
            print(f"\n📊 COUNTRIES WITH MEDIUM CYBERSECURITY RELEVANCE ({len(medium_relevance)})")
            print("="*60)
            for _, row in medium_relevance.iterrows():
                print(f"• {row['country']} (Confidence: {row.get('confidence', 'N/A')})")
else:
    print("No relevance level data found to analyze.")


🔥 COUNTRIES WITH HIGH CYBERSECURITY RELEVANCE (0)
No countries with HIGH relevance found in this assessment.

📊 COUNTRIES WITH MEDIUM CYBERSECURITY RELEVANCE (1)
• Albania (Confidence: ConfidenceLevel.HIGH)


In [ ]:
# # Show countries with HIGH cybersecurity relevance (uncomment if you ran the full assessment)
# if 'processed_full' in locals() and 'relevance_level' in processed_full.columns:
#     high_relevance = processed_full[processed_full['relevance_level'] == 'high']
#     print(f"\n🔥 COUNTRIES WITH HIGH CYBERSECURITY RELEVANCE ({len(high_relevance)})")
#     print("="*60)
#     for _, row in high_relevance.iterrows():
#         print(f"• {row['country']} (Confidence: {row.get('confidence', 'N/A')})")
#         if 'explanation' in row and row['explanation']:
#             explanation = row['explanation'][:100] + "..." if len(row['explanation']) > 100 else row['explanation']
#             print(f"  └─ {explanation}")
#         print()

## Configuration & Usage

### Switching Between Test and Full Mode

**Test Mode** (`TEST_MODE = True`):
- Runs on first 5 countries only
- Quick validation of setup and functionality
- Generates `ministry_ministry_cyber_test.*` output files

**Full Mode** (`TEST_MODE = False`):
- Runs on all 193 countries
- Takes 3-4 minutes to complete
- Generates `ministry_ministry_cyber_full_assessment.*` output files

### Key Features

1. **✅ Script Integration**: Uses the `cyber_relevance_assessment.py` script as an importable module
2. **✅ Dataset Loading**: Loads the ministry ministries dataset with 193 countries
3. **✅ Structured Assessment**: Uses Pydantic models for reliable data extraction
4. **✅ Unified Code**: Same assessment code for both test and full modes, controlled by `max_queries`
5. **✅ Results Processing**: Expands structured data into analyzable columns
6. **✅ Multiple Output Formats**: Saves results in JSON, JSONL, and CSV formats

### Key Columns Extracted:
- `country`: Country name
- `domain`: Ministry domain ("ministry")
- `research_question`: The assessment question asked
- `enriched_citations`: Citations with metadata
- `relevance_level`: HIGH/MEDIUM/LOW/NONE cybersecurity involvement
- `confidence`: Assessment confidence level
- `explanation`: Detailed explanation with citations

### To Run Full Assessment:
1. Change `TEST_MODE = False` in the second cell
2. Run all cells - the same code will automatically handle the full dataset